In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [6]:
import requests

from bs4 import BeautifulSoup

In [8]:

c = pd.read_csv(r'C:\Users\Fujitsu\Desktop\BLR_PoastalCode.csv')
c.head()

,Postal Code,Neighborhood,Division,Latitude,Longitude
0,560001,Bangalore G.P.O.,Bangalore North,12.9914,77.5944
1,560001,Legislators Home,Bangalore North,12.9914,77.5944
2,560001,Vasanthanagar,Bangalore North,12.9914,77.5944
3,560001,Mahatma Gandhi Road,Bangalore North,12.9914,77.5944
4,560001,Vidhana Soudha,Bangalore North,12.9914,77.5944


In [9]:
c.shape

(278, 5)

In [10]:
#import pandas as pd

#b = pd.read_csv(r'C:\Users\Fujitsu\Desktop\Zone.csv')
#b.head()

In [11]:
#c = pd.merge(a,b, on= 'Pin')
#c.head(10)

In [12]:
c.dropna(subset=["Postal Code","Neighborhood","Division","Latitude","Longitude"], how='any',inplace= True)
c.head(10)

,Postal Code,Neighborhood,Division,Latitude,Longitude
0,560001,Bangalore G.P.O.,Bangalore North,12.9914,77.5944
1,560001,Legislators Home,Bangalore North,12.9914,77.5944
2,560001,Vasanthanagar,Bangalore North,12.9914,77.5944
3,560001,Mahatma Gandhi Road,Bangalore North,12.9914,77.5944
4,560001,Vidhana Soudha,Bangalore North,12.9914,77.5944
5,560001,Rajbhavan (Bangalore),Bangalore North,12.9914,77.5944
6,560001,HighCourt,Bangalore North,12.9914,77.5944
7,560001,Cubban Road,Bangalore North,12.9914,77.5944
8,560001,Bangalore Bazaar,Bangalore North,12.9914,77.5944
9,560001,CMM Court Complex,Bangalore North,12.9914,77.5944


In [13]:
c.describe

<bound method NDFrame.describe of      Postal Code         Neighborhood         Division  Latitude  Longitude
0         560001     Bangalore G.P.O.  Bangalore North   12.9914    77.5944
1         560001     Legislators Home  Bangalore North   12.9914    77.5944
2         560001        Vasanthanagar  Bangalore North   12.9914    77.5944
3         560001  Mahatma Gandhi Road  Bangalore North   12.9914    77.5944
4         560001       Vidhana Soudha  Bangalore North   12.9914    77.5944
..           ...                  ...              ...       ...        ...
273       562157            Doddajala  Bangalore North   13.1438    77.6169
274       562157          Bettahalsur  Bangalore North   13.1438    77.6169
275       562157           Chikkajala  Bangalore North   13.1438    77.6169
276       562162    Madanayakanahalli  Bangalore North   17.8647    77.8229
277       562162           Dasanapura  Bangalore North   17.8647    77.8229

[278 rows x 5 columns]>

In [14]:
c.shape

(278, 5)

In [16]:
#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim

In [17]:
#!pip install geocoder

In [15]:
CLIENT_ID = 'XUAYS5LROHLIGN0LTDNAPHS015AYOFWTTLX45U05EUFZ4ZJK' # your Foursquare ID
CLIENT_SECRET = 'NEC2M2AKOEHNOS11MFKJEQHNNLCYRFMZSC521A40JGIRBJIY' # your Foursquare Secret
VERSION = '20200531' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XUAYS5LROHLIGN0LTDNAPHS015AYOFWTTLX45U05EUFZ4ZJK
CLIENT_SECRET:NEC2M2AKOEHNOS11MFKJEQHNNLCYRFMZSC521A40JGIRBJIY


In [16]:
c.loc[0, 'Neighborhood']

'Bangalore G.P.O.'

In [17]:
neighborhood_latitude = c.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = c.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = c.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bangalore G.P.O. are 12.9914, 77.5944.


In [18]:
LIMIT = 100

radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XUAYS5LROHLIGN0LTDNAPHS015AYOFWTTLX45U05EUFZ4ZJK&client_secret=NEC2M2AKOEHNOS11MFKJEQHNNLCYRFMZSC521A40JGIRBJIY&v=20200531&ll=12.9914,77.5944&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee5db135f782d7169f38d86'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Vasanth Nagar',
  'headerFullLocation': 'Vasanth Nagar, Bangalore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 12.995900004500005,
    'lng': 77.59900959038852},
   'sw': {'lat': 12.986899995499996, 'lng': 77.58979040961147}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6ee375f964a52029cf2ce3',
       'name': 'Ujwal Bar & Restaurant',
       'location': {'address': '402-A, Millers Road, Vasanthnagar,',
        'lat': 12.99227960861634,
        'lng': 77.59447349833962,
        'labeledLatLngs': [{'label'

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ujwal Bar & Restaurant,Indian Restaurant,12.992280,77.594473
1,Millers 46,Steakhouse,12.991666,77.594207
2,Alliance Française,Concert Hall,12.991232,77.596723
3,Infinitea,Tea Room,12.987157,77.594835
4,Imperial Restaurant,Indian Restaurant,12.991150,77.593837


In [22]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

32 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Place', 
                  'Place Latitude', 
                  'Place Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:

B_venues = getNearbyVenues(names=c['Neighborhood'],
                                   latitudes=c['Latitude'],
                                   longitudes=c['Longitude']
                                  )

Bangalore G.P.O.
Legislators Home
Vasanthanagar
Mahatma Gandhi Road
Vidhana Soudha
Rajbhavan (Bangalore)
HighCourt
Cubban Road
Bangalore Bazaar
CMM Court Complex
Dr. Ambedkar Veedhi
Bangalore City
Bangalore Corporation Building
Sri Jayachamarajendra Road
Palace Guttahalli
Malleswaram
Swimming Pool Extn
Vyalikaval Extn
Pampamahakavi Road
Thyagarajnagar
Mavalli
Basavanagudi
Fraser Town
J.C.Nagar
Training Command IAF
Agram
Air Force Hospital
Hulsur Bazaar
H.A.L II Stage
K. G. Road
Bangalore Dist Offices Bldg
Rajajinagar
Industrial Estate (Bangalore)
Rajajinagar IVth Block
Jayangar III Block
Madhavan Park
Science Institute
Jalahalli
Jalahalli East
Jalahalli West
Krishnarajapuram R S
Doorvaninagar
Ramamurthy Nagar
Vimanapura
NAL
Chamrajpet (Bangalore)
Gaviopuram Extension
Seshadripuram
Gayathrinagar
Srirampuram
Yeshwanthpur Bazar
Yeswanthpura
Magadi Road
Anandnagar (Bangalore)
Hebbal Kempapura
H.A. Farm
Museum Road
Richmond Town
CMP Centre And School
Deepanjalinagar
Governmemnt Electric Fac

In [26]:
print(B_venues.shape)
B_venues.head()

(3098, 7)


,Place,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bangalore G.P.O.,12.9914,77.5944,Ujwal Bar & Restaurant,12.992280,77.594473,Indian Restaurant
1,Bangalore G.P.O.,12.9914,77.5944,Millers 46,12.991666,77.594207,Steakhouse
2,Bangalore G.P.O.,12.9914,77.5944,Alliance Française,12.991232,77.596723,Concert Hall
3,Bangalore G.P.O.,12.9914,77.5944,Infinitea,12.987157,77.594835,Tea Room
4,Bangalore G.P.O.,12.9914,77.5944,Imperial Restaurant,12.991150,77.593837,Indian Restaurant


In [27]:
B_venues.groupby('Place').count()

,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Place,,,,,,
A F Station Yelahanka,1,1,1,1,1,1
Adugodi,4,4,4,4,4,4
Agara,41,41,41,41,41,41
Agram,1,1,1,1,1,1
Air Force Hospital,1,1,1,1,1,1
...,...,...,...,...,...,...
Yelachenahalli,1,1,1,1,1,1
Yelahanka,3,3,3,3,3,3
Yemalur,56,56,56,56,56,56


In [28]:
# one hot encoding
B_onehot = pd.get_dummies(B_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
B_onehot['Neighborhood'] = B_venues['Place'] 

# move neighborhood column to the first column
fixed_columns = [B_onehot.columns[-1]] + list(B_onehot.columns[:-1])
B_onehot = B_onehot[fixed_columns]

B_onehot.head()

,Zoo Exhibit,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,BBQ Joint,...,Tennis Stadium,Thai Restaurant,Tibetan Restaurant,Toll Plaza,Tourist Information Center,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
B_onehot.shape

(3098, 143)

In [30]:
B_grouped = B_onehot.groupby('Neighborhood').mean().reset_index()
B_grouped

,Neighborhood,Zoo Exhibit,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Garage,...,Tennis Stadium,Thai Restaurant,Tibetan Restaurant,Toll Plaza,Tourist Information Center,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,A F Station Yelahanka,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
1,Adugodi,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
2,Agara,0.0,0.000000,0.02439,0.02439,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.02439,0.000000,0.0,0.0,0.0,0.000000,0.02439,0.0,0.02439
3,Agram,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
4,Air Force Hospital,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Yelachenahalli,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
228,Yelahanka,0.0,0.333333,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.333333,0.00000,0.0,0.00000
229,Yemalur,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.017857,0.0,...,0.0,0.00000,0.017857,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00000
230,Yeshwanthpur Bazar,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,1.0,0.0,0.000000,0.00000,0.0,0.00000


In [31]:
B_grouped.shape

(232, 143)

In [32]:
#Sort Data by Most Popular Venues
num_top_venues = 5

for hood in B_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = B_grouped[B_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A F Station Yelahanka----
                venue  freq
0         IT Services   1.0
1           Multiplex   0.0
2  Miscellaneous Shop   0.0
3   Mobile Phone Shop   0.0
4     Motorcycle Shop   0.0


----Adugodi----
                     venue  freq
0        Indian Restaurant  0.25
1     Fast Food Restaurant  0.25
2            Design Studio  0.25
3              Bus Station  0.25
4  North Indian Restaurant  0.00


----Agara----
               venue  freq
0               Café  0.20
1  Indian Restaurant  0.15
2                Pub  0.07
3        Coffee Shop  0.07
4     Ice Cream Shop  0.05


----Agram----
                venue  freq
0        Home Service   1.0
1         Zoo Exhibit   0.0
2           Multiplex   0.0
3  Miscellaneous Shop   0.0
4   Mobile Phone Shop   0.0


----Air Force Hospital----
                venue  freq
0        Home Service   1.0
1         Zoo Exhibit   0.0
2           Multiplex   0.0
3  Miscellaneous Shop   0.0
4   Mobile Phone Shop   0.0


----Amruthahalli----
    

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = B_grouped['Neighborhood']

for ind in np.arange(B_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(B_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F Station Yelahanka,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
1,Adugodi,Fast Food Restaurant,Indian Restaurant,Design Studio,Bus Station,Diner,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
2,Agara,Café,Indian Restaurant,Coffee Shop,Pub,Lounge,Ice Cream Shop,Yoga Studio,Bar,Paper / Office Supplies Store,Mexican Restaurant
3,Agram,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
4,Air Force Hospital,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop


In [35]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [36]:
#Group The Data Into Clusters
kclusters = 8

B_grouped_clustering = B_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(B_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 6, 6, 1, 3, 7, 1, 1])

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

B_merged = c

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
B_merged = B_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

B_merged.head() # check the last columns!

,Postal Code,Neighborhood,Division,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560001,Bangalore G.P.O.,Bangalore North,12.9914,77.5944,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
1,560001,Legislators Home,Bangalore North,12.9914,77.5944,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
2,560001,Vasanthanagar,Bangalore North,12.9914,77.5944,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
3,560001,Mahatma Gandhi Road,Bangalore North,12.9914,77.5944,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
4,560001,Vidhana Soudha,Bangalore North,12.9914,77.5944,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub


In [38]:
address = "Bangalore, BLR"

geolocator = Nominatim(user_agent="bangalore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore city are 13.196697, 77.70758655918868.


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(B_merged['Latitude'], B_merged['Longitude'], B_merged['Neighborhood'], B_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        B_merged['Latitude'], 
        B_merged['Longitude'], 
        B_merged['Neighborhood'], 
        B_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 0, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Benson Town,0.0,Café,Intersection,Asian Restaurant,Bakery,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop
207,Tarabanahalli,0.0,Auto Garage,Café,Business Service,Asian Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
216,C.V.Raman Nagar,0.0,Auto Garage,Café,Business Service,Asian Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
217,ISRO Anthariksha Bhavan,0.0,Auto Garage,Café,Business Service,Asian Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
218,R.M.V. Extension II Stage,0.0,Auto Garage,Café,Business Service,Asian Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
219,Koramangala VI Bk,0.0,Auto Garage,Café,Business Service,Asian Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar


In [53]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 1, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangalore G.P.O.,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
1,Legislators Home,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
2,Vasanthanagar,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
3,Mahatma Gandhi Road,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
4,Vidhana Soudha,1.0,Indian Restaurant,Coffee Shop,Chinese Restaurant,Tea Room,Italian Restaurant,Bakery,Pizza Place,Mediterranean Restaurant,Pool Hall,Pub
...,...,...,...,...,...,...,...,...,...,...,...,...
271,Hunasamaranahalli,1.0,Indian Restaurant,Coffee Shop,Lake,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
272,Tarahunise,1.0,Indian Restaurant,Coffee Shop,Lake,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
273,Doddajala,1.0,Indian Restaurant,Coffee Shop,Lake,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
274,Bettahalsur,1.0,Indian Restaurant,Coffee Shop,Lake,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar


In [55]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 2, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
128,Subramanyapura,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
129,Chikkalasandra,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
130,Doddakallasandra,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
131,Konanakunte,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
132,Tataguni,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
133,A F Station Yelahanka,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
134,BSF Campus Yelahanka,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
142,G.K.V.K.,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
147,Devanagundi,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store
148,Kadugodi,2.0,IT Services,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop,Fishing Store


In [44]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 3, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Yeshwanthpur Bazar,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
51,Yeswanthpura,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
53,Anandnagar (Bangalore),3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
54,Hebbal Kempapura,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
55,H.A. Farm,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
59,Deepanjalinagar,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio
60,Governmemnt Electric Factory,3.0,Tourist Information Center,Yoga Studio,Dessert Shop,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner,Design Studio


In [45]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 4, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,Mundur,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
104,Bidrahalli,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
105,Thambuchetty palya,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
106,Virgonagar,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
107,Bhattarahalli,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
153,Hongasandra,4.0,Bakery,Yoga Studio,Fishing Store,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner


In [46]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 5, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
194,Lingarajapuram,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
195,Kacharakanahalli,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
196,St. Thomas Town,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
252,Yadavanahalli,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
253,Attibele,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
254,Bidaraguppe,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
255,Mayasandra,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
256,Neralur,5.0,Indian Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop


In [47]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 6, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Fraser Town,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
23,J.C.Nagar,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
24,Training Command IAF,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
25,Agram,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
26,Air Force Hospital,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
27,Hulsur Bazaar,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
28,H.A.L II Stage,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
29,K. G. Road,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
30,Bangalore Dist Offices Bldg,6.0,Home Service,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop


In [48]:
# Examining Clusters
B_merged.loc[B_merged['Cluster Labels'] == 7, B_merged.columns[[1] + list(range(5, B_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
138,Attur,7.0,ATM,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
152,Begur,7.0,ATM,Herbs & Spices Store,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar
243,Anekal,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
244,Vanakanahalli,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
245,Thammanayakanahalli,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
246,Bestamaranahalli,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
247,Sidihoskote,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
248,Guddanahalli,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
249,Indalavadi,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner
250,Marsur,7.0,Business Service,ATM,Yoga Studio,Fast Food Restaurant,Farmers Market,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Diner


In [68]:
## It is noticable that Cluster 1 is the most viable clusters to start a business. The proximity to various Neighborhood is an advantage for businessmen. Options are wide open since it filled with Restaurants, coffee shops, Market area, Pubs, Halls...One csan thinkof starting Childrens play area, or another restuarant etc.. simillarly Cluster 3's common spot is Tourism so one can start connect business plan in this cluster. So objective is to see business opportunity and each cluster has unique opportunity to think of.